In [ ]:
import json
import logging
import time
import numpy as np

from betfairstreamer.models.market_cache import MarketCache
from betfairstreamer.models.order_cache import OrderCache

from betfairstreamer.api_client import BetfairAPIClient
from betfairstreamer.helpers.stream_helpers import (
    create_market_subscription,
    create_order_subscription
)

# Setup options for logging and prettier printing.
from betfairstreamer.stream.betfair_connection_pool import BetfairConnectionPool

np.set_printoptions(precision=3)
logging.basicConfig(level=logging.INFO)


USERNAME="betfair username"
PASSWORD="betfair password"
APP_KEY="betfair app key"
CERT_CRT_PATH="path to .crt file, relative or absolute"
CERT_KEY_PATH="path to .key file, relative or absolute"

# Add locale here if you're are in any of
# these jurisdictions [italy (IT), spain (ES), romania (RO), sweden (SE)]
# i.e. LOCALE="SE"

LOCALE=""

api_client = BetfairAPIClient.from_requests_backend(
    username=USERNAME,
    password=PASSWORD,
    app_key=APP_KEY,
    cert_crt_path=CERT_CRT_PATH,
    cert_key_path=CERT_KEY_PATH,
    locale=LOCALE
)

### Create subscriptions.

Specify which market one want to subscribe to.

References.
[MarketSubscription](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Exchange+Stream+API#ExchangeStreamAPI-MarketSubscriptionMessage),
[OrderSubscription](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Exchange+Stream+API#ExchangeStreamAPI-OrderSubscriptionMessage).

In [ ]:
soccer_subscription_message = create_market_subscription(
    # Soccer
    event_type_ids=["1"],
    market_types=["MATCH_ODDS"],
    country_codes=["BY", "DE"],
    # Type of data you want back
    fields=[
        "EX_MARKET_DEF",
        "EX_BEST_OFFERS",
        "EX_LTP",
        "EX_BEST_OFFERS_DISP",
        "EX_ALL_OFFERS"
    ],
    # Set delay in ms
    conflate_ms=0
)

order_subscription_message = create_order_subscription()

### Cache
Betfair sends partial update (deltas), each cache merges all updates from respective stream

In [ ]:
market_cache = MarketCache()
order_cache = OrderCache()

### Connection Pool
For each subscription message, __BetfairConnectionPool__ will create a separate stream to betfair.

In [ ]:
# Get session token, used as authentication to Betfair.
session_token = api_client.get_session_token()

connection_pool = BetfairConnectionPool.create_connection_pool(
    subscription_messages=[soccer_subscription_message, order_subscription_message],
    app_key=APP_KEY,
    session_token=session_token
)

![img](selections.png)

We have a market with __three__ selections/runners (rows). Shakhter, Dinamo and The Draw.

There are two sides Back/Lay separated  by color.

Each side has three ladder levels (columns).

__MarketBook__ represent all of these as a numpy ndarray,
in this case a ndarray with axes (3, 2, 3, 2), (selection, side, ladder_level, price/size).

*  first axis (row) is the runner index (0/1/2),

* second axis side (color) to select Back/Lay (0/1),

* third axis ladder level (columns) (0, 1, 2)

* fourth axis price and size (0/1)

#### Example

If i want the __best__ (ladder 0) __back__ __price__ for __Shakhter__
```python
market_book.best_display[0, 0, 0, 0]
```
If i want the best __lay__ price for __The draw__
```python
market_book.best_display[2, 1, 0, 0]
```

If i want second best back prices for all selections
```python
market_book.best_display[:, 0, 1, 0]
```

In [ ]:
# Before running this I strongly recommend to right click on the cell and select "Enable scrolling for outputs", there could be alot of updates :)
try:
    # Read a single message from the stream.
    for update in connection_pool.read():
        # Decode update from bytes to dict
        update = json.loads(update)

        # Update each cache
        market_updates = market_cache(update)
        order_updates = order_cache(update)

        # iterate through all market books that received an update
        for market_book in market_updates:
            print(f"{market_book.market_id} "
                  # The delay in seconds from the time betfair sent the message
                  # until one processed it.
                  f"{round(time.time() - market_cache.publish_time/1000, 4)} "
                  # Show all back price and size for runner 0
                  f"{market_book.best_display[0, 0, :, 0]}")

        # iterate through all orders that were updated
        for order in order_updates:
            print(order)
finally:
    # Must be here. Jupyter does not "close" sockets automatically,
    # which will hold streams open to betfair and will
    # take forever to close.
    connection_pool.close()